In [ ]:
!pip install transformers tokenizers datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
input_filename = "naslovi_new.input"
dataset_filename = input_filename + '.prepared'
pretrained_model_name = 'gpt2'
new_model_name = pretrained_model_name + '_3'

In [4]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="/content/drive/MyDrive/ML-Vesti/naslovi-simple-tokenizer.json")
tokenizer.pad_token = "<pad>"


In [5]:
from datasets import load_dataset
# datasets = load_dataset('text', data_files=dataset_filename, split="train[90%],validate")
eval, train = load_dataset('text', data_files='/content/drive/MyDrive/ML-Vesti/' + dataset_filename, split=['train[:10%]', 'train[10%:]'])
def prepare_for_trainer(example):
  tokenized = tokenizer(example['text'], truncation=True, padding='max_length', max_length=20)
  tokenized["labels"] = tokenized["input_ids"].copy()
  return tokenized
tokenized_train = train.map(prepare_for_trainer, num_proc=2, remove_columns=["text"])
tokenized_eval = eval.map(prepare_for_trainer, num_proc=2, remove_columns=["text"])


Using custom data configuration default-9b16dadd29989af1


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-9b16dadd29989af1/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.
 

In [6]:
from transformers import Trainer, AutoConfig, AutoModelForCausalLM, TrainingArguments

config = AutoConfig.from_pretrained(pretrained_model_name)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name)
# model.to('cuda:0')
training_args = TrainingArguments(
    # new_model_name,
    output_dir='/content/drive/MyDrive/ML-Vesti/' + new_model_name,
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    save_strategy='epoch',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_total_limit=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    # prediction_loss_only=True,
    tokenizer=tokenizer,
)


In [7]:
import logging
logger = logging.getLogger(__name__)
logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )

Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False


In [ ]:
# import gc
# import torch 
# model = None
# trainer = None
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
train_result = trainer.train()
trainer.save_model('/content/drive/MyDrive/ML-Vesti/modeli/' + new_model_name)

***** Running training *****
  Num examples = 1968703
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1230440


Epoch,Training Loss,Validation Loss
1,3.170000,3.767735
2,3.041900,3.622925
3,2.946000,3.545960


***** Running Evaluation *****
  Num examples = 218745
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/ML-Vesti/gpt2_3/checkpoint-246088
Configuration saved in /content/drive/MyDrive/ML-Vesti/gpt2_3/checkpoint-246088/config.json
Model weights saved in /content/drive/MyDrive/ML-Vesti/gpt2_3/checkpoint-246088/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ML-Vesti/gpt2_3/checkpoint-246088/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/ML-Vesti/gpt2_3/checkpoint-246088/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 218745
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/ML-Vesti/gpt2_3/checkpoint-492176
Configuration saved in /content/drive/MyDrive/ML-Vesti/gpt2_3/checkpoint-492176/config.json
Model weights saved in /content/drive/MyDrive/ML-Vesti/gpt2_3/checkpoint-492176/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ML-Vesti/gpt2_3/checkpoint-492176/

In [ ]:
import numpy as np
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/ML-Vesti/modeli/' + new_model_name)
model.config.pad_token_id = 1

for temp in np.linspace(0.2, 2, 10):
    print()
    print("###### Temperature: " + str(temp))
    for i in range(0, 5):
        out = model.generate(do_sample=True, temperature=temp)[0]
        print(tokenizer.decode(out, skip_special_tokens=True))